### (Execute if your running in Binder)

In [ ]:
# Uncomment these lines

# !pip install pandas
# !pip install openpyxl
# !pip install networkx

# HS Conversion Tables

In [1]:
import pandas as pd
import os
from pathlib import Path
import platform
import networkx as nx

### En desarrollo

In [162]:
import pandas as pd
import functools
from datetime import datetime
import numpy as np
import networkx as nx
from operator import and_


class HSCorrelations: 
    
    def __init__(self, path=None):
        self.__HS = ['HS92', 'HS96', 'HS02', 'HS07', 'HS12', 'HS17'] #should be updated if changed 
        print(f"Loading HS Correlation Tables from path provided")
        if path: 
            self.__data = pd.read_excel(path,
                            usecols= self.__HS,
                            dtype = {v:'object' for v in self.__HS}).dropna().drop_duplicates()
        else:
            print("Loading HS Correlation Tables from UNSTATS\n server")
            self.__url = "https://unstats.un.org/unsd/trade/classifications/tables/CompleteCorrelationsOfHS-SITC-BEC_20170606.xlsx"
            self.__data = pd.read_excel(self.__url,
                                usecols= self.__HS,
                                dtype = {v:'object' for v in self.__HS}).dropna().drop_duplicates()
        print("Tables already loaded. Use `available_methods()` for more information\n")
        
    def available_methods(self):
        print("Use `get_df()` to get dataframe with the correlations between all HS versions\n")
        print("Use `get_codes()` to get the codes by: version (default HS17, str or list) or chapter") 
        #uncomplete
     
    def get_df(self):
        return self.__data
    
    def get_url(self):
        return self.__url
    
    def get_versions(self): 
        return self.__HS

    def get_codes(self, version="HS17", chapter=None): 
        df = pd.DataFrame(self.__data[version], columns = [version]).drop_duplicates()
        if chapter:
            df = df[df[version].str.startswith(chapter)]
        return df
    
    def check_position(self, position=None):
        if position:
            return position in self.__data.values
            
    
    def return_used_versions(self, start_year=None, end_year=None):
        if start_year==None: 
            start_year = 1992
        if end_year==None: 
            end_year = datetime.now().year
        if start_year < 1992 or end_year<start_year or end_year>datetime.now().year: 
            print("Uncorrect years")
        years = [1992, 1996, 2002, 2007, 2012, 2017]
        version_dict = {k:v for k,v in zip(years, self.__HS)}
        start_v = functools.reduce(lambda a,b: a if a<=start_year else b, list(reversed(years)))
        end_v = functools.reduce(lambda a,b: a if a<=end_year else b, list(reversed(years)))
        years_list = years[years.index(start_v):years.index(end_v)+1]
        return [version_dict[x] for x in years_list]
    
    def filter_df(self, positions=None, start_year=None, end_year=None): 
        
        cols = self.return_used_versions(start_year, end_year)
        df = self.__data[cols]
        if positions:
            filters = [(df[col].isin(positions)) for col in cols]
            mask = functools.reduce(and_, filters, True)
            return df[mask].drop_duplicates().reset_index(drop=True)
        else:
            return df         
            
    def find_homogeneous_serie(self, position=None, start_year=None, end_year=None):
        if self.check_position(position): 
            cols = self.return_used_versions(start_year, end_year)
            data = self.filter_df(start_year, end_year)
            versions = [v for v in cols if data[v].str.contains(position).any()]
            for col in cols:
                data[col] = data[col].apply(lambda x: col+"-"+str(int(x)).zfill(6))
            connections = []
            for i in range(len(cols)-1): 
                temp_tup_list = list(data[[cols[i], cols[i+1]]].drop_duplicates().itertuples(index=False, name=None))
                connections = connections + temp_tup_list 
            G = nx.Graph()
            G.add_edges_from(connections)
            positions = sorted(nx.node_connected_component(G, versions[-1]+"-"+position))
            return positions
        else:
            print("Please define a correct position")
    
    def genSankey(self, position=None, start_year=None, end_year=None, output_title='Sankey Diagram'):
        if self.check_position(position): 
            related = list(set([x[5:] for x in self.find_homogeneous_serie(position, start_year, end_year)]))
            cat_cols = self.return_used_versions(start_year, end_year)
            df = self.filter_df(related, start_year, end_year)
            df['count'] = 1
            for col in cat_cols: 
                df[col] = df[col].apply(lambda x: col+"-"+str(int(x)).zfill(6))
            colorPalette = ['#4B8BBE','#306998','#FFE873','#FFD43B','#646464',"#002060"]
            labelList = []
            colorNumList = []
            for catCol in cat_cols:
                labelListTemp =  list(set(df[catCol].values))
                colorNumList.append(len(labelListTemp))
                labelList = labelList + labelListTemp

            # remove duplicates from labelList
            labelList = list(dict.fromkeys(labelList))

            # define colors based on number of levels
            colorList = []
            for idx, colorNum in enumerate(colorNumList):
                colorList = colorList + [colorPalette[idx]]*colorNum

            # transform df into a source-target pair
            for i in range(len(cat_cols)-1):
                if i==0:
                    sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],'count']]
                    sourceTargetDf.columns = ['source','target','count']
                else:
                    tempDf = df[[cat_cols[i],cat_cols[i+1],'count']]
                    tempDf.columns = ['source','target','count']
                    sourceTargetDf = pd.concat([sourceTargetDf,tempDf])

            sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()

            # add index for source-target pair
            sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
            sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))

            # creating the sankey diagram
            data = dict(
                type='sankey',
                node = dict(
                  pad = 15,
                  thickness = 20,
                  line = dict(
                    color = "black",
                    width = 0.5
                  ),
                  label = labelList,
                  color = colorList
                ),
                link = dict(
                  source = sourceTargetDf['sourceID'],
                  target = sourceTargetDf['targetID'],
                  value = sourceTargetDf['count']
                )
              )

            layout =  dict(
                title = output_title,
                font = dict(
                  size = 15
                )
            )

            fig = dict(data=[data], layout=layout)
            return fig
        else: 
           print("Please define a correct position") 
    
    def recursive_trade_off(df, position, k): 
    
        if k>=0:
            currcols = df.columns.tolist()[k:]
            print(f"Evaluating graph object with {','.join(currcols)} versions\n")
            data = df[currcols].drop_duplicates()
            connections = []
            for i in range(len(currcols)-1): 
                temp_tup_list = list(data[[currcols[i], currcols[i+1]]].drop_duplicates().itertuples(index=False, name=None))
                connections = connections + temp_tup_list
            G = nx.Graph()
            G.add_edges_from(connections)
            try:
                positions = sorted(nx.node_connected_component(G, currcols[-1]+"-"+position))
                if len(positions)==len(currcols): 
                    return recursive_trade_off(df, position, k-1)
                else: 
                    print("Evaluation finished\n")
                    print(f"The {','.join(currcols[1:])} versions support your position with no precision loss")
            except:
                print(f"Position {position} not founded")
        else:
            print(f"The {','.join(df.columns.tolist())} versions support your position with no precision loss")
        
    def trade_off(self, position=None, start_year=None, end_year=None):
        if start_year==None: 
            start_year = 1992
        if end_year==None: 
            end_year = datetime.now().year
        if start_year < 1992 or end_year<start_year or end_year>datetime.now().year: 
            print("Uncorrect years")

        if self.ckeck_position(position): 
            versions = self.return_used_versions(start_year, end_year)
            print(f"Period between {start_year} and {end_year} contains the {','.join(versions)} versions\n") 
            print(f"Loading HS Correlations Tables\n")
            versions = [v for v in versions if self.__data[v].str.contains(position).any()]

            if len(versions)>0: 
                print(f"The position {position} was included in the {','.join(versions)} versions\n")
                data = self.__data[versions].drop_duplicates()
                for col in data.columns:
                    data[col] = data[col].apply(lambda x: col+"-"+str(int(x)).zfill(6))
                k=len(versions)-2
                print(f"Evaluating maximum period with no precision loss for position {position}\n")
                return recursive_trade_off(data, position, k)
            else:
                print(f"Position {position} not founded in that period")
        else:
            print("Please define a correct position")

In [163]:
fullpath = 'C:\\Users\\Usuario\\Desktop\\CEP\\HSConversionTables\\data\\CompleteCorrelationsOfHS-SITC-BEC_20170606.xlsx'
a = HSCorrelations(path = fullpath)

Loading HS Correlation Tables from path provided
Tables already loaded. Use `available_methods()` for more information



In [153]:
print(a.check_position("010130"))

True


In [123]:
a.get_codes(version="HS92", chapter="02")

,HS92
67,020110
68,020120
69,020130
70,020210
71,020220
72,020230
73,020311
74,020312
75,020319
76,020321


In [122]:
a.get_versions()

['HS92', 'HS96', 'HS02', 'HS07', 'HS12', 'HS17']

In [55]:
a.return_used_versions(2006,2022)

['HS02', 'HS07', 'HS12', 'HS17']

In [62]:
a.filter_df(["010111"], 1993, 1996)

,HS92,HS96
0,010111,010111
1,010111,010111


In [131]:
a.find_homogeneous_serie("010130")

['HS02-010110',
 'HS02-010190',
 'HS07-010110',
 'HS07-010190',
 'HS12-010121',
 'HS12-010129',
 'HS12-010130',
 'HS12-010190',
 'HS17-010121',
 'HS17-010129',
 'HS17-010130',
 'HS17-010190',
 'HS92-010111',
 'HS92-010119',
 'HS92-010120',
 'HS96-010111',
 'HS96-010119',
 'HS96-010120']

In [168]:
import plotly.graph_objects as go
import chart_studio.plotly as py
import plotly

fig = a.genSankey("010130")
if fig is not None:
    plotly.offline.plot(fig, validate=False)

In [126]:
a.trade_off("010130", 2018, 2019) #corregir que devuelva los años posibles

Period between 2018 and 2019 contains the HS17 versions

Loading HS Correlations Tables

The position 010130 was included in the HS17 versions

Evaluating maximum period with no precision loss for position 010130

The HS17 versions support your position with no precision loss
